# Dashboard about covid 19 situation in Vietnam

## Import libraries

In [36]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

app = dash.Dash(external_stylesheets=[dbc.themes.CYBORG])

## Read and process data

In [37]:
Cases_Deaths_URL = 'https://covid19.who.int/WHO-COVID-19-global-data.csv'
Total_Stat_URL = 'https://covid19.who.int/WHO-COVID-19-global-table-data.csv'
Vaccine_URL = 'https://covid19.who.int/who-data/vaccination-data.csv'

Stat_CD = pd.read_csv(Cases_Deaths_URL, usecols=['Country','New_cases', 'New_deaths', 'Date_reported'])
Stat_CD = Stat_CD[Stat_CD['Country']=='Viet Nam']

Total = pd.read_csv(Total_Stat_URL)
Total = Total.loc['Viet Nam']

vaccine = pd.read_csv(Vaccine_URL)
vaccine = vaccine[vaccine['COUNTRY'] == 'Viet Nam']
vaccine = vaccine.reset_index()

## Set up image, color

In [38]:
image1 = 'https://d18lkz4dllo6v2.cloudfront.net/cumulus_uploads/entry/2020-04-06/COVID%20.jpg?w=660'
image2 = "https://www.sanofi.com.vn/-/media/Project/One-Sanofi-Web/Websites/Asia-Pacific/Sanofi-VN/Home/thong-tin-bao-chi/covid-19-nhan-manh-gia-tri-quan-trong-cua-viec-cham-soc-ban-than/Article-block-Covid-19.jpg?la=vi&hash=9763B0A5603D3FED373554950C0F6854"
colors = {
    'Purple' : '#A427F2',
    'Cyan': '#00FFFF',
    'White' : '#FFFFFF',
    'Blue' : '#0000FF',
    'Silver' : '#C0C0C0',
}

## Page header

In [39]:
team_member = [
    dbc.Col(html.H6('Nguyen Phuong Thao Vy', style={'color' : colors['Blue']}), width='auto'),
    dbc.Col(html.H6('Vu Duy Tung', style={'color' : colors['Blue']}), width='auto'),
    dbc.Col(html.H6('Ta Viet Thang', style={'color' : colors['Blue']}), width='auto'),
    dbc.Col(html.H6('Chau Minh Khai', style={'color' : colors['Blue']}), width='auto')
]

def generate_page_header():
    header1 = dbc.Row(
        [
            dbc.Col(html.H4(
                'Welcome to COMP1010 project',
                style={'textAlign':'center', 'color':colors['Cyan']}
            ))
        ],
    )

    header2 = dbc.Row(
        [
            dbc.Col(html.H3(
                'A Dashboard about Covid-19 Situation in Vietnam',
                style={'textAlign':'center', 'color':colors['Cyan']}
            ))
        ],
    )
    header3 = dbc.Row(
        team_member,
        justify='center',
        style={
            'backgroundColor' : colors['Silver']
        }
    )
    header = [header1, header2, header3]
    return header

## Enter your email

In [40]:
def Input_email():
    return html.Div(
        [
            dbc.Row(
                [
                    dbc.Col(html.H5(
                        [
                            "Enter your email:   ",
                            dcc.Input(id='my-input', value='Alibaba@vinuni.edu.vn', type='email')
                        ],
                        style={'textAlign':'center', 'color':colors['Purple']}
                    )),
                ]
            ),
            dbc.Row(
                [
                    dbc.Col(html.H5(id='my-output', style={'textAlign': 'center', 'color' : colors['Purple']}))
                ]
            ),

        ]
    )

## Generate Cards for Total Cases and Total Deaths

In [41]:
def Card_with_stat(name, image, num, color):
    if( color == 'Cyan' ): BG = 'danger'
    else: BG = 'warning'
    card = dbc.Card(
        [
            dbc.CardImg(src=image, top=True),
            dbc.CardBody(
                [
                    html.H3(
                        name,
                        style={
                            'textAlign':'center',
                            'color':colors[color]
                        }
                    ),
                    html.H4(
                        '{:,}'.format(num),
                        style={
                            'textAlign':'center',
                            'color':colors[color]
                        }
                    ),
                ]
            ),
        ],
        style={'width' : '18rem'},
        color=BG,
        inverse='True'
    )
    return card

def Generate_card():
    return dbc.Row(
        [
            dbc.Col(Card_with_stat('Total cases', image1, Total['WHO Region'], 'Blue'), width='auto'),
            dbc.Col(Card_with_stat('Total deaths', image2, Total['Cases - newly reported in last 24 hours'], 'Cyan'), width='auto'),
        ],
        align='center',
        justify='center'
    )


## Generate table

In [42]:
def Table():
    new_vac = Stat_CD.iloc[-10:]
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(new_vac[['Date_reported','New_cases','New_deaths']].columns),
                    line_color='darkslategray',
                    fill_color='royalblue',
                    align='center',
                    font=dict(color='white', size=17),
                    height=35
        ),
        cells=dict(values=[new_vac.Date_reported, new_vac.New_cases, new_vac.New_deaths],
                   line_color='darkslategray',
                   fill_color='paleturquoise',
                   align='center',
                   font=dict(color='black', size=17),
                   height=25
        ))
    ])
    return fig

def Generate_table():
    return html.Div([
        dcc.Graph(
            id = 'Table',
            figure = Table().update_layout(
                template='plotly_dark',
                plot_bgcolor='rgba(0, 0, 0, 0)',
                paper_bgcolor='rgba(0, 0, 0, 0)',
            )
        )
    ])


## Generate line graph with button to change the status

In [43]:
def check_list():
    return dcc.RadioItems(
        id='check_list',
        options=[{'label': i, 'value': i} for i in ['New_cases', 'New_deaths']],
        value='New_cases',
        labelStyle={'display': 'block'},
        style={"padding": "10px", "max-width": "800px", "margin": "auto"},
    )

def Cases_Deaths_LineGraph(cases_or_deaths, y_title):
    fig = px.line(Stat_CD, x = Stat_CD.Date_reported, y = cases_or_deaths, title='Daily cases in Vietnam', height=500, color_discrete_sequence =['maroon'], markers=True)
    fig.update_layout(title_x=0.5, xaxis_title="Date", yaxis_title=y_title)
    fig.update_layout(
        template='plotly_dark',
        paper_bgcolor='rgba(0, 0, 0, 0)',
    )
    return fig

def graph1():
    return dcc.Graph(id='graph1', figure=Cases_Deaths_LineGraph(Stat_CD.New_cases, 'Number of cases'))

## Generate bar chart

In [44]:
def Cases_Deaths_BarChart():
    fig = px.bar(Stat_CD, x = Stat_CD.Date_reported, y = [Stat_CD.New_deaths, Stat_CD.New_cases] * 719, barmode='group', height=500)
    fig.update_layout(
        template='plotly_dark',
        paper_bgcolor='rgba(0, 0, 0, 0)',
    )
    return fig

def bar_chart():
    return dcc.Graph(id='bar_chart', figure=Cases_Deaths_BarChart())

## Generate pie chart

In [45]:
def Vaccine_Pie_Chart():
    Full_Dose = vaccine.loc[0]['PERSONS_FULLY_VACCINATED']
    OnePlus_Dose = vaccine.loc[0]['PERSONS_VACCINATED_1PLUS_DOSE']

    valuee = [Full_Dose, OnePlus_Dose - Full_Dose]
    namee = ['People who got 1 dose', 'People fully vaccinated']

    fig = px.pie(values=valuee, names=namee)
    return fig

def Generate_Pie_Chart():
    return dcc.Graph(
        figure=Vaccine_Pie_Chart().update_layout(
            template='plotly_dark',
            plot_bgcolor='rgba(0, 0, 0, 0)',
            paper_bgcolor='rgba(0, 0, 0, 0)',
        )
    )

## Generate card for vaccine information

In [46]:
def Card_Vaccine(header, text):
    if header == 'Total Vaccination':
        COLOR = "danger"
    if header == 'Vaccine used':
        COLOR = "success"
    if header == 'Number vaccine types used':
        COLOR = "warning"

    card_head_style = {'textAlign' : 'center', 'fontSize' : '150%'}

    if header == 'Vaccine used':
        card_body_style = {'textAlign' : 'center', 'fontSize' : '40%'}
    else:
        card_body_style = {'textAlign': 'center', 'fontSize': '150%'}

    return dbc.Card(
        [
            dbc.CardHeader(header , style=card_head_style),
            dbc.CardBody([
                html.H5(text),
            ], style=card_body_style)
        ],
        color=COLOR,
        inverse=True
    )

## Show all vaccine data

In [47]:
def Vaccine_Info():
    return html.Div([
        dbc.Row(
            [
                dbc.Col(Generate_Pie_Chart(), width={'size': 12})
            ],
            align='center',
            justify='center'
        ),
        dbc.Row(
            dbc.Col(
                dbc.CardGroup(
                    [
                        Card_Vaccine(
                            header='Total Vaccination',
                            text='{:,}'.format(vaccine.loc[0]['TOTAL_VACCINATIONS'])
                        ),
                        Card_Vaccine(
                            header='Vaccine used',
                            text=vaccine.loc[0]['VACCINES_USED']
                        ),
                        Card_Vaccine(
                            header='Number vaccine types used',
                            text=str(vaccine.loc[0]['NUMBER_VACCINES_TYPES_USED'])
                        )
                    ]
                )
            )
        )
    ])

## Generate cards for read/watch news

In [48]:
def New(link, title, text, button, iimage):
    return dbc.Card(
        [
            dbc.CardImg(src = iimage, top = True),
            dbc.CardBody(
                [
                    html.H5(title),
                    html.H6(text),
                    dbc.Button(button, href=link, color='primary')
                ]
            )
        ],
        color='warning',
        outline=True,
    )

def News_with_card():
    return dbc.Row(
        [
            dbc.Col(New(
                link='https://www.youtube.com/watch?v=wT2m3kljcSU',
                title='Fake news',
                text='The explosion of coranavirus also leads to the raise of fake new on social medias',
                button='Lets watch',
                iimage='https://image.vietnamlawmagazine.vn/uploadvietnamlaw/2021/7/23/fake_newsjpg172654812.jpg'
            ), width={'size':3}),
            dbc.Col(New(
                link='https://www.youtube.com/watch?v=qQUdy5ZXAqo',
                title='Deadliest pandemics in History (till 2020)',
                text='Plagues and epidemics have ravaged humanity throughout its existence, often changing the course of history',
                button='Lets see',
                iimage='https://i.ytimg.com/vi/qQUdy5ZXAqo/maxresdefault.jpg'
            ), width={'size':3}),
            dbc.Col(New(
                link='https://www.usnews.com/news/health-news/articles/2021-12-10/pandemic-mystery-scientists-focus-on-covids-animal-origins?fbclid=IwAR0dUFwC6fUZsLL4WnJquaR6ysYADavPEMKNcri_oOkyMMHfluQixS7e7nA',
                title='Scientists Focus on COVID Animal Origins',
                text='Most scientists believe it emerged in the wild and jumped from bats to humans, either directly or through another animal. Others theorize it escaped from a Chinese lab.',
                button='Read this',
                iimage='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRQyk2DAWarOLrYFlcwYxApLJj09WZZsIc_dQ&usqp=CAU'
            ), width={'size':3}),
            dbc.Col(New(
                link='https://e.vnexpress.net/news/news/vietnam-falls-to-bottom-of-bloomberg-covid-resilience-ranking-4407673.html?fbclid=IwAR1xsPiXIPV-9hnCuW2NUMXPyOH0BjWpZCcikd8XaTRzZZmGc3if0rVTTBM',
                title='Viet Nam',
                text='Vietnam falls to bottom of Bloomberg Covid resilience ranking',
                button='Why',
                iimage='https://file1.dangcongsan.vn/data/0/images/2021/06/14/chamsocytepv/covid-khauhieu.jpg?dpi=150&quality=100&w=680'
            ), width={'size':3}),
        ],
        justify='center',
        align='center'
    )

## Final layout

In [49]:
def Title(sentence):
    return dbc.Row(
        [
            dbc.Col(html.H5(
                sentence,
                style={'color': colors['Purple']}), width='auto')
        ],
        align='center',
        justify='center'
    )

def generate_layout():
    header = generate_page_header()
    layout = dbc.Container(
        [
            header[0],
            header[1],
            header[2],
            html.Br(),
            Input_email(), html.Hr(),
            Generate_card(), html.Hr(),
            Title('Statistics for the numbers of new cases and new deaths'),
            dbc.Row(
                [
                    dbc.Col(Generate_table())
                ],
            ),
            html.Hr(),
            Title('The fluctuates in the numbers of new cases and new deaths'),
            dbc.Row(
                [
                    dbc.Col(check_list(), width='auto'),
                    dbc.Col(graph1())
                ],
                align='center'
            ),
            html.Hr(),
            Title('Bar chart to compare between new cases and new deaths'),
            dbc.Row(
                [
                    dbc.Col(bar_chart()),
                ],
                align='center',
                justify='center'
            ),
            html.Hr(),
            Title('Vaccine Information'),
            Vaccine_Info(),
            html.Hr(),
            Title('Read/Watch more about Covid-19'), html.Br(),
            News_with_card(),
        ],
        fluid=True,
    )

    return layout

## Make your website interactive

In [50]:
@app.callback(
    [Output('my-output', 'children'),
     Output('graph1', 'figure')],
    [Input('my-input', 'value'),
    Input('check_list', 'value')]
)
def update_output_div(input_value1, input_value2):
    your_name = ''
    for i in input_value1:
        if i == '@': break
        your_name += i

    if input_value2 == 'New_cases':
        fig = Cases_Deaths_LineGraph(Stat_CD.New_cases, 'Number of cases')
    else:
        fig = Cases_Deaths_LineGraph(Stat_CD.New_deaths, 'Number of deaths')

    return f'WELCOME "{your_name.upper()}" TO OUR WEBSITE', fig

## Run the server

In [ ]:
app.layout = generate_layout()

if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
